# IMPORTS

In [ ]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [ ]:
import json
import os
import matplotlib.pyplot as plt
from AxonDeepSeg.testing.segmentation_scoring import *
#from ipywidgets.widgets import *

In [ ]:
from scipy.misc import imread, imsave

# 1/ Build the dataset for training.

In [ ]:
# input data to build the training set
dataset_name = 'SEM_3classes_reduced'
gps = 0.2

In [ ]:
path_data = '../data/' + dataset_name + '/raw/'
path_testing = '../data/SEM_3classes_reduced/testing/pilou/'
#path_testing = '../data/images_nyu_tem/'

# output path of training data path
path_training = '../data/' + dataset_name + '/training/'

# 2/ Load the config file and train the network

Choose here the name of the model you want to load

In [ ]:
#model_name = 'cv_3c_d4_c2_k3__0-4560'
model_name = 'baseline_3c_balanced_nobn-1957'

In [ ]:
# optional input path of a model to initialize the training
#path_model_init = 'network_testing/test_2905'
path_model_init = None

# output path for trained U-Net
path_model = '../models/' + model_name

path_configfile = path_model + '/config_network.json'

if not os.path.exists(path_model):
    os.makedirs(path_model)

with open(path_configfile, 'r') as fd:
    config_network = json.loads(fd.read())
    
# OPTIONAL : specify the gpu one wants to use.
gpu_device = 'gpu:0' # or gpu_device = 'gpu:1' these are the only two possible targets for now.

# 3/ Apply the model to segment one image

#### Segmentation

In [ ]:
from AxonDeepSeg.apply_model import axon_segmentation
axon_segmentation(path_testing, path_model, config_network, crop_value=25, general_pixel_size=gps)

# 4/ Visualisation of the segmentation

Import images

In [ ]:
imorg = imread(path_testing + '/image.png', flatten=True)
imads = imread(path_testing + '/AxonDeepSeg.png', flatten=True)

Display original image

In [ ]:
plt.figure(figsize=(13,10))
plt.title('Original image')
plt.imshow(imorg, cmap='gray')
plt.show();

Display segmentation and compare it with original image

In [ ]:
plt.figure(figsize=(13,10))
plt.title('Segmented image')
plt.imshow(imads)
plt.show();

In [ ]:
plt.figure(figsize=(13,10))
plt.title('Superposed images')
plt.imshow(imorg, cmap='gray', alpha=0.5)
plt.imshow(imads, cmap='viridis', alpha=0.5)

# 5/ Metrics analysis

We are now going to analyze each class dice. First we compute masks for each prediction and each ground truth

In [ ]:
mask = imread(path_testing + '/mask.png', flatten=True)
pred = imread(path_testing + '/AxonDeepSeg.png', flatten=True)

Creating the mask necessary to compute the dice

In [ ]:
gt_axon = mask > 200
gt_myelin = np.logical_and(mask >= 50, mask <= 200)

pred_axon = pred > 200
pred_myelin = np.logical_and(pred >= 50, pred <= 200)

Affichage de chaque classe

In [ ]:
plt.figure(figsize=(13,10))
plt.subplot(2,2,1)
plt.imshow(pred_axon, cmap='gray')
plt.title('Axon prediction')
plt.subplot(2,2,2)
plt.imshow(gt_axon, cmap='gray')
plt.title('Axon ground truth')
plt.subplot(2,2,3)
plt.imshow(pred_myelin, cmap='gray')
plt.title('Myelin prediction')
plt.subplot(2,2,4)
plt.imshow(gt_myelin, cmap='gray')
plt.title('Myelin ground truth')
plt.show();

### Computing the dice for axon and myelin

In [ ]:
dice_axon = pw_dice(pred_axon, gt_axon)
dice_myelin = pw_dice(pred_myelin, gt_myelin)

print 'Dice for Axon : ' + str(dice_axon)
print 'Dice for myelin : ' + str(dice_myelin)

In [ ]:
score_analysis(imorg, gt_axon, pred_axon)

In [ ]:
imorg.shape

In [ ]:
data_axon_dice = dice(imorg, gt_axon, pred_axon, min_area=4)

In [ ]:
data_axon_dice.describe()